In [52]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width' , 1000)

In [53]:
# Variables
season = "2020-21"

In [54]:
all_players = pd.read_csv (r'./data/' + season + '/Raw_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Raw_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Raw_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Raw_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Raw_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Raw_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Raw_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



## Rename columns

In [56]:
player_gameweek_history.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,2,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,50,0,12184,0,0
3,2,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,50,-4015,8234,51,4066
4,3,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,55,0,69564,0,0


In [57]:
player_gameweek_history.rename(columns = {"element" : "player_id" , "opponent_team" : "opponent_team_id" , "fixture" : "fixture_id" } , inplace = True)
all_players.rename(columns = {"code" : "player_code", "element_type" : "player_type_id" , "id" : "player_id" , "team" : "team_id" , "points_per_game" : "PPG"} , inplace=True)
all_teams.rename(columns = {"id" : "team_id" , "code" : "team_code" , "name" : "team_name" , "short_name" : "team_short_name"} , inplace = True)
all_events.rename(columns = {"id" : "gameweek_id"} , inplace = True)
player_types.rename(columns = {"id" : "player_type_id" , "singular_name" : "position_name" , "singular_name_short" : "position_name_short"} , inplace = True)
game_phases.rename(columns = {"id" : "phase_id"} , inplace = True)
player_season_history.rename(columns = {"element_code" : "player_code"} , inplace = True)
player_future_fixture.rename(columns = {"id" : "fixture_id" , "team_h" : "home_team_id" , "team_a" : "away_team_id" , "events" : "gameweek_id" , "code" : "fixture_code"} , inplace = True)
#player_gameweek_history.rename(columns = {} , inplace = True)


## Drop columns

In [58]:
all_events.head(5)

,gameweek_id,name,deadline_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,highest_score,is_previous,is_current,is_next,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2020-09-12 10:00:00+00:00,50,True,True,4761681.0,1599904800,0,142.0,True,False,False,"[{'chip_name': 'bboost', 'num_played': 112843}...",259.0,1.0,254.0,"{'id': 254, 'points': 20}",0,4.0,4.0
1,2,Gameweek 2,2020-09-19 10:00:00+00:00,59,True,True,6234344.0,1600509600,0,165.0,False,True,False,"[{'chip_name': 'bboost', 'num_played': 94615},...",259.0,302.0,390.0,"{'id': 390, 'points': 24}",14637421,4.0,254.0
2,3,Gameweek 3,2020-09-26 10:00:00+00:00,0,False,False,NaN,1601114400,0,NaN,False,False,True,[],NaN,NaN,NaN,NaN,0,NaN,NaN
3,4,Gameweek 4,2020-10-03 10:00:00+00:00,0,False,False,NaN,1601719200,0,NaN,False,False,False,[],NaN,NaN,NaN,NaN,0,NaN,NaN
4,5,Gameweek 5,2020-10-17 12:30:00+00:00,0,False,False,NaN,1602937800,0,NaN,False,False,False,[],NaN,NaN,NaN,NaN,0,NaN,NaN


In [59]:
all_events.drop(columns = ["deadline_time_epoch"] , inplace = True)
player_types.drop(columns = ["plural_name" , "plural_name_short"] , inplace = True)
all_players.drop(columns = ["squad_number" , "special" , "transfers_in" , "transfers_out" , "first_name" , "second_name"] , inplace = True)

## Change date columns

In [60]:
all_players['news_added'] = all_players['news_added'].dt.tz_localize(None)
all_events['deadline_time'] = all_events['deadline_time'].dt.tz_localize(None)
player_future_fixture['kickoff_time'] = player_future_fixture['kickoff_time'].dt.tz_localize(None)

player_gameweek_history['kickoff_time'] = pd.to_datetime(player_gameweek_history['kickoff_time'])
player_gameweek_history['kickoff_time'] = player_gameweek_history['kickoff_time'].dt.tz_localize(None)


## Add Columns

In [61]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id
0,23,2128310,11,NaN,1,NaN,3.0,False,0,False,2020-09-28 19:00:00,Gameweek 3,False,5,1
1,29,2128316,1,NaN,15,NaN,4.0,False,0,False,2020-10-04 13:00:00,Gameweek 4,True,3,1
2,44,2128331,12,NaN,1,NaN,5.0,False,0,False,2020-10-17 14:00:00,Gameweek 5,False,5,1
3,49,2128336,1,NaN,9,NaN,6.0,False,0,False,2020-10-24 14:00:00,Gameweek 6,True,3,1
4,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-10-31 15:00:00,Gameweek 7,False,4,1


In [62]:
def opponent_team(row):
    if row['is_home'] == True:
        val = row['away_team_id']
    elif row['is_home'] == False:
        val = row['home_team_id']
    return val

def team(row):
    if row['is_home'] == False:
        val = row['away_team_id']
    elif row['is_home'] == True:
        val = row['home_team_id']
    return val


player_future_fixture['opponent_team_id'] = player_future_fixture.apply(opponent_team,axis=1)
player_future_fixture['team_id'] = player_future_fixture.apply(team,axis=1)


## Calculate New Columns

In [64]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'player_code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code', 'total_points', 'transfers_in_event', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_text', 'direct_freekicks_order',
     

In [65]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,ep_this,event_points,form,player_id,in_dreamteam,news,news_added,now_cost,photo,PPG,selected_by_percent,status,team_id,team_code,total_points,...,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,37605,0,0,-1,1,0,3,-0.5,0.5,0,0.0,1,False,NaN,NaT,69,37605.jpg,0.0,0.9,a,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,536,217,537,219,534,215,537,218,4.0,NaN,NaN,NaN,NaN,NaN
1,75.0,0.0,39476,-1,1,-1,1,0,2,-0.4,0.0,0,0.0,2,False,Calf injury - 75% chance of playing,2020-09-11 11:00:08.600094,49,39476.jpg,0.0,0.1,d,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,419,165,411,165,375,154,426,165,NaN,NaN,NaN,NaN,NaN,NaN
2,100.0,100.0,41270,0,0,0,0,0,2,0.0,1.0,1,0.5,3,False,NaN,2020-09-10 14:30:11.632386,55,41270.jpg,1.0,0.9,a,1,3,1,...,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,487,187,490,187,475,184,494,187,NaN,NaN,4.0,NaN,NaN,NaN
3,NaN,NaN,54694,0,0,0,0,0,3,5.5,6.5,5,6.0,4,False,NaN,NaT,120,54694.jpg,6.0,36.6,a,1,3,12,...,0,0,1,0,0,0,39,64.6,51.7,69.0,18.5,31,15,19,15,16,6,13,9,NaN,NaN,6.0,NaN,1.0,NaN
4,75.0,NaN,58822,0,0,-1,1,0,2,-0.4,0.5,0,0.0,5,False,Calf injury - 75% chance of playing,2020-09-23 09:00:14.881983,49,58822.jpg,0.0,0.4,d,1,3,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,423,166,415,166,380,155,430,166,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
all_players['PP90'] = np.divide(all_players['total_points'] , all_players['minutes'])*90 

In [67]:
all_players['PPMM'] = np.divide(all_players['PPG'] , all_players['now_cost']/10)

In [68]:
all_players['VAPM'] = np.divide(all_players['PPG']-2 , all_players['now_cost']/10)

In [75]:
all_players[['web_name' , 'player_type_id', 'total_points' , 'minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['minutes']>100].sort_values(by='VAPM' , ascending = False).head()

,web_name,player_type_id,total_points,minutes,PP90,PPG,PPMM,VAPM
245,Castagne,2,23,180,11.500000,11.5,2.053571,1.696429
257,Bamford,4,21,130,14.538462,10.5,1.842105,1.491228
183,Calvert-Lewin,4,25,158,14.240506,12.5,1.736111,1.458333
260,Costa,3,19,180,9.500000,9.5,1.696429,1.339286
156,Zaha,3,23,180,11.500000,11.5,1.619718,1.338028


## Save cleaned data

In [71]:
all_players.to_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', index = False, header=True)
all_teams.to_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv', index = False, header = True)
all_events.to_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', index = False, header = True)
player_types.to_csv(r'./data/' + season + '/Cleaned_data/player_types.csv', index = False, header = True)
game_phases.to_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv', index = False, header = True)

player_season_history.to_csv (r'./data/' + season + '/Cleaned_data/player_season_history.csv', index = False, header=True)
player_gameweek_history.to_csv (r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv', index = False, header=True)
player_future_fixture.to_csv (r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', index = False, header=True)